In [35]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, BooleanType, IntegerType, StringType, LongType, DoubleType
import nltk
import twarc
from tableschema import Table

In [36]:
spark = SparkSession.builder.master('local').appName('local').getOrCreate()

In [37]:
LABELED_PATH = '../data/processed/full-tweets-labeled/'

LDA_PATH = '../data/lda/'
LDA_FILE = 'LDA_Output.csv'

In [38]:
df_schema = StructType([
    StructField('id', LongType(), True),
    StructField('prediction', DoubleType(), True),
    StructField('weighted_label', DoubleType(), True),
])

lda_schema = StructType([
    StructField('id', LongType(), True),
    StructField('topic_index', StringType(), True),
    StructField('topic_label', StringType(), True),
])

tableau_schema = StructType([
    StructField('id', LongType(), True),
    StructField('date', StringType(), True),
    StructField('label', IntegerType(), True),
   
    StructField('score_naive', DoubleType(), True),
    StructField('score_evo', DoubleType(), True),
    
    StructField('topic', StringType(), True),
    StructField('topic_lda', StringType(), True),
    
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    
    StructField("isUS", BooleanType(), True),
    StructField("state", StringType(), True),
])

In [40]:
df = spark.read.csv(LABELED_PATH + '/' + '*.csv', header=True, schema=df_schema)
df_lda = spark.read.csv(LDA_PATH + '/' + '*.csv', header=False, schema=lda_schema)
df.printSchema()
df_lda.printSchema()

df_tableau = df.join(lda, on=['id'])

df_tableau.show()
# df_tableau = df_tableau.join()

#df_tableau = spark.createDataFrame(spark.sparkContext.emptyRDD(),tableau_schema)
#df_tableau.printSchema()


# Create table
# table = Table(, schema='../data/schema.json')

# Print schema descriptor
# print(table.schema.descriptor)

# Print cast rows in a dict form
# for keyed_row in table.iter(keyed=True):
#     print(keyed_row)

root
 |-- id: long (nullable = true)
 |-- prediction: double (nullable = true)
 |-- weighted_label: double (nullable = true)

root
 |-- id: long (nullable = true)
 |-- topic_index: string (nullable = true)
 |-- topic_label: string (nullable = true)

+-------------------+----------+-------------------+-----------+---------------+
|                 id|prediction|     weighted_label|topic_index|    topic_label|
+-------------------+----------+-------------------+-----------+---------------+
|1246892082888945666|       0.0|0.34590212711893237|    Topic 1|          Other|
|1246892725158449152|       1.0| 0.9765624810792524|    Topic 1|          Other|
|1246894604307312640|       1.0| 0.9851553133346029|    Topic 2|Social Distance|
|1246894744174759950|       0.0| 0.2760483246626915|    Topic 3|          Masks|
|1246895626236964873|       0.0|0.14001506904275743|    Topic 4|       Lockdown|
|1246895919536246785|       0.0|0.17449963481495315|    Topic 1|          Other|
|1246897071942250497|